# 2020 Target Data Preparation

**Info Material**

Number of districts per state: https://ballotpedia.org/Population_represented_by_state_legislators

## Unzipping all files

In [46]:
import zipfile
import os
path = "drive/MyDrive/US Elections/individual_states"

for file in os.listdir(path):
  filepath = f"{path}/{file}/{file}.csv.zip"
  with zipfile.ZipFile(filepath,"r") as zip_ref:
      zip_ref.extractall(f"{path}/{file}")

# Read in data on precinct level

In this part the data has to be aggregated from precinct to district level.

In [47]:
# Create list of csv filenames
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

path = "drive/MyDrive/US Elections/individual_states"
files = []
for folder in os.listdir(path):
  files.append(f"{path}/{folder}/{folder}.csv")

# Create list of dataframe names
df_names = []
for file in files:
  df_names.append(file[-23:-21])

dfs = {}
for df, file in zip(df_names, files):
    dfs[df] = pd.read_csv(file, low_memory=False)

In [48]:
%%time
data = pd.concat(dfs.values(), ignore_index=True)

CPU times: user 3.86 s, sys: 1.18 s, total: 5.05 s
Wall time: 5.08 s


In [ ]:
data[data["state"] == "ARIZONA"]["office"].value_counts()

RETENTION COURT OF APPEALS                 34206
CORPORATION COMMISSIONER                   26802
RETENTION ARIZONA SUPREME COURT JUSTICE    26802
STATE HOUSE                                14577
US PRESIDENT                               13401
US HOUSE                                    8934
US SENATE                                   8934
PROPOSITION 207                             8934
PROPOSITION 208                             8934
STATE SENATE                                7566
Name: office, dtype: int64

In [ ]:
data.shape

(10394316, 25)

In [ ]:
state_house = data.copy()

In [ ]:
def data_wrangling(df):
  # Take only instances of state house of representatives elections
  df = df[df["office"] == "STATE HOUSE"]
  # Delete all informations about under- and overvoting
  df.drop(df[df["candidate"].isin(["OVERVOTES", "UNDERVOTES"])].index, inplace=True)
  # Write-in candidates and NA's are of No Party Affiliation
  df.loc[df[df["party_detailed"].isna()].index, ["party_detailed", "party_simplified"]] = "No Party Affiliation"
  return df

In [ ]:
%%time
state_house = data_wrangling(data)

CPU times: user 1.01 s, sys: 12.8 ms, total: 1.03 s
Wall time: 1.03 s


In [ ]:
state_house.shape

(518947, 25)

In [ ]:
pd.set_option("display.max_columns", None)
state_house.head()

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,district,magnitude,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
16156,01-446 AURORA,STATE HOUSE,DEMOCRAT,DEMOCRAT,ELECTION DAY,248.0,NaN,NaN,DISTRICT 1,2001.0,CHRISTOPHER QUIST,001,1,STATE,2020,GEN,ALASKA,False,False,AK,2,94,81,2020-11-03,False
16157,01-446 AURORA,STATE HOUSE,REPUBLICAN,REPUBLICAN,ELECTION DAY,495.0,NaN,NaN,DISTRICT 1,2001.0,"BARTON S ""BART"" LEBON",001,1,STATE,2020,GEN,ALASKA,False,False,AK,2,94,81,2020-11-03,False
16158,01-455 FAIRBANKS NO. 1,STATE HOUSE,DEMOCRAT,DEMOCRAT,ELECTION DAY,45.0,NaN,NaN,DISTRICT 1,2001.0,CHRISTOPHER QUIST,001,1,STATE,2020,GEN,ALASKA,False,False,AK,2,94,81,2020-11-03,False
16159,01-455 FAIRBANKS NO. 1,STATE HOUSE,REPUBLICAN,REPUBLICAN,ELECTION DAY,90.0,NaN,NaN,DISTRICT 1,2001.0,"BARTON S ""BART"" LEBON",001,1,STATE,2020,GEN,ALASKA,False,False,AK,2,94,81,2020-11-03,False
16160,01-465 FAIRBANKS NO. 2,STATE HOUSE,DEMOCRAT,DEMOCRAT,ELECTION DAY,88.0,NaN,NaN,DISTRICT 1,2001.0,CHRISTOPHER QUIST,001,1,STATE,2020,GEN,ALASKA,False,False,AK,2,94,81,2020-11-03,False


In [ ]:
cat_columns = state_house.select_dtypes(exclude=["number"]).columns
state_house[cat_columns] = state_house[cat_columns].applymap(lambda x:x.lower() if type(x) == str else x)
state_house.head()

,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,district,magnitude,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
16156,01-446 aurora,state house,democrat,democrat,election day,248.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
16157,01-446 aurora,state house,republican,republican,election day,495.0,NaN,NaN,district 1,2001.0,"barton s ""bart"" lebon",001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
16158,01-455 fairbanks no. 1,state house,democrat,democrat,election day,45.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
16159,01-455 fairbanks no. 1,state house,republican,republican,election day,90.0,NaN,NaN,district 1,2001.0,"barton s ""bart"" lebon",001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
16160,01-465 fairbanks no. 2,state house,democrat,democrat,election day,88.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False


## Check district count

According to Ballotpedia there are 4828 house districts, where a total of 5411 representatives get elected. The elections of those seats aren't all at the same time. Although 2020 was the presidential election and most of the seats are allocated in that election, I expect the number to be a little lower.

Ballotpredia Link: https://ballotpedia.org/State_Legislative_Districts

In [ ]:
state_house.precinct.nunique()

142233

In [ ]:
state_house.groupby("state")["district"].nunique().sum()

3906

In [ ]:
state_house.groupby(["precinct", "state"])["state"].nunique().sum()

147993

In [ ]:
state_house.groupby(["precinct", "state"])["state"].nunique().sum()

147993

## Check for number of states

In [ ]:
state_house.groupby("state")["district"].nunique().sum()

3906

In [ ]:
print(state_house.state.nunique())
print(state_house.state.sort_values().unique())

43
['alaska' 'arizona' 'arkansas' 'california' 'colorado' 'connecticut'
 'delaware' 'florida' 'georgia' 'hawaii' 'idaho' 'illinois' 'iowa'
 'kansas' 'kentucky' 'maine' 'massachusetts' 'michigan' 'minnesota'
 'missouri' 'montana' 'nevada' 'new hampshire' 'new jersey' 'new mexico'
 'new york' 'north carolina' 'north dakota' 'ohio' 'oklahoma'
 'pennsylvania' 'rhode island' 'south carolina' 'south dakota' 'tennessee'
 'texas' 'utah' 'vermont' 'virginia' 'washington' 'west virginia'
 'wisconsin' 'wyoming']


**missing states (7)** = Alabama, Indiana, Louisiana, Maryland, Mississippi, Nebraska, Oregon

## Data Cleaning Steps

* Clean wrong party infos to either democrats or repbulicans
* Change local democratic parties to "democrat"
* Summarize all indpendent candidates under the same "party" -> independent
* Delete Conservative and Libertarian candidates as they win too few seats
* Check for candidates -> delete all blank ballots, under-/overvoted entries
* Delete all other parties than the three to predict
* Delete districts which elect more than one representative

In [ ]:
df_info = pd.DataFrame()
df_info["Data Type"] = state_house.dtypes
df_info["Missing Values"] = state_house.isna().sum()
df_info["No. Unique Values"] = state_house.nunique()
df_info

,Data Type,Missing Values,No. Unique Values
precinct,object,0,142233
office,object,0,1
party_detailed,object,0,62
party_simplified,object,0,6
mode,object,0,16
votes,float64,0,4271
county_name,object,1170,1506
county_fips,float64,1170,2408
jurisdiction_name,object,0,13793
jurisdiction_fips,float64,0,7252


#### Democratic-Repbulican

In [ ]:
state_house[state_house["party_detailed"].isin(["democratic/republican", "republican-democrat", "democrat-republican"])].candidate.unique()

array(['mark longietti', 'doyle heffley', 'linda joy sullivan',
       'eileen "lynn" dickinson', 'brian k savage', 'james gregoire',
       'brian smith', 'michael j marcotte', 'woodman h "woody" page',
       'mark a higley', 'charles "butch" shaw', 'jim harrison',
       'francis "topper" mcfaun', 'rob laclair'], dtype=object)

Democrats: <br>
MARK LONGIETTI, LINDA JOY SULLIVAN, BRIAN SMITH 

Republican <br>
DOYLE HEFFLEY, EILEEN "LYNN" DICKINSON, BRIAN K SAVAGE, JAMES GREGOIRE, MICHAEL J MARCOTTE, WOODMAN H "WOODY" PAGE, MARK A HIGLEY, CHARLES "BUTCH" SHAW, JIM HARRISON, FRANCIS "TOPPER" MCFAUN, ROB LACLAIR

In [ ]:
state_house.loc[state_house[state_house["candidate"].isin(["mark longietti", "linda joy sullivan", "brian smith"])].index, "party_detailed"] = "democrat"
state_house.loc[state_house[state_house["candidate"].isin(
    ["doyle heffley", 'eileen "lynn" dickinson', 'brian k savage', 'james gregoire', 'mark a higley', 'michael j marcotte', 'woodman h "woody" page',
     'charles "butch" shaw', 'jim harrison', 'francis "topper" mcfaun', 'rob laclair'])].index, "party_detailed"] = "republican"

#### Local democratic parties to democrats

In [ ]:
# Progressive party of Vermont
state_house.loc[state_house[state_house["party_detailed"].isin(
    ["democrat-progressive", "progressive-democrat"])].index, "party_detailed"] = "democrat"

# Rise and unite of New York
state_house.loc[state_house[state_house["party_detailed"] == "rise and unite"].index, "party_detailed"] = "democrat"

# Democratic Nonpartisan League Party of North Dakota
state_house.loc[state_house[state_house["party_detailed"] == "democratic-npl"].index, "party_detailed"] = "democrat"

# Democratic Farmer Labor = Democratic party in Minnesota
state_house.loc[state_house[state_house["party_detailed"] == "democratic farmer labor"].index, "party_detailed"] = "democrat"

In [ ]:
#state_house[state_house["party_detailed"] == "independent american"]

#### All Independents and Nonpartisan to Independents

In [ ]:
state_house.loc[
  state_house[state_house["party_detailed"].isin(
      ["nonpartisan", "independent", "independent for maine", "independence", "independent american", "no party affiliation"]
      )].index, "party_detailed"] = "independent"

#### Delete Conservative and Liberatarian Party members

Conservative and Libertarian party members won each one seat in the state houses. That's too little to be included.

In [ ]:
state_house.drop(state_house[state_house["party_detailed"].isin(["conservative", "libertarian"])].index, inplace=True)

#### Check for candidates

In [ ]:
state_house.candidate.value_counts()[:50]

writein                     38133
blank ballots                3275
blanks                       2214
all others                   2213
over                         1940
rick herrick                 1872
thurston smith               1872
colin j schmitt              1383
kevin a cahill               1377
jim wood                     1342
charlotte svolos             1342
michaelle c solages          1260
joseph j sackman iii         1064
michael a montesano          1064
cecilia aguiar-curry         1044
matthew l nelson             1044
edward p ra                  1024
james c ramos                1010
jennifer tullius             1010
gina l sillitti               944
ragini srivastava             944
charles w cole                942
steve bennett                 942
sarita bhandarkar             922
andrew a monteleone           920
charles d lavine              920
melissa l miller              912
david g mcdonough             896
chad mayes                    888
andrew f kotyu

candidates to drop:
* BLANKS, NONE, BLANK, SPOILED, OVER, BLANK BALLOTS

In [ ]:
state_house.drop(state_house[state_house["candidate"].isin(
    ["blank", "blanks", "blank ballots", "none", "spoiled", "over"])].index, inplace=True)

#### Delete all other parties than Dem, Rep, Ind

In [ ]:
state_house["party_detailed"].unique()

array(['democrat', 'republican', 'independent', 'alaska independence',
       "veteran's party", 'independent nomination',
       'kamal hammouda for iowa house', 'constitution', 'green',
       'green independent', 'for the people', 'common sense independent',
       'unenrolled', 'socialist resurgence', 'epic', 'working families',
       'veterans party of america', 'legal marijuana now',
       'grassroots legalize cannabis', 'green-rainbow',
       'working class party', 'us taxpayers party', 'natural law party',
       'pro-gun pro-life', 'unity of colorado', 'aloha aina',
       'american shopping', 'united party',
       'abolitionist society pennsylvania party', 'utah united',
       'alliance', 'labor', "seattle people's", 'the alliance',
       'mountain', 'progressive', 'berlin-northfield alliance',
       'vets for vets', 'safe neighborhoods', 'save our city',
       'covid19 stories', 'justice & peace', 'serve america movement',
       'schley for 70 assy', 'liberal', 'pro

In [ ]:
state_house.shape

(476857, 25)

In [ ]:
state_house.drop(
    state_house[~state_house["party_detailed"].isin(["democrat", "republican", "independent"])].index,
    inplace=True)

#### Delete districts with more than one representative

In [ ]:
state_house.magnitude.value_counts()

1     433805
2      20129
3        364
4        227
5         66
11        46
10        22
9         19
8         17
7         15
6         13
Name: magnitude, dtype: int64

In [ ]:
magnitude_over1 = state_house[state_house["magnitude"] > 1].groupby(["state", "district"])["votes"].sum()

In [ ]:
magnitude_over1

state    district 
arizona  001          225667.0
         002          124719.0
         003           97514.0
         004          102457.0
         005          132154.0
                        ...   
vermont  wdh-4          6733.0
         wdr-1          8232.0
         wdr-3-2        7381.0
         wdr-4-2        6904.0
         wdr-ora-2      9632.0
Name: votes, Length: 176, dtype: float64

In [ ]:
state_house.drop(state_house[state_house["magnitude"] > 1].index, inplace=True)

#### State house winners per district

In [ ]:
test_house_district = state_house.groupby(["state", "district", "party_detailed"])["votes"].sum()
test_house_district

state    district  party_detailed
alaska   001       democrat          3027.0
                   republican        3769.0
         002       democrat          1565.0
                   republican        3631.0
         003       republican        7001.0
                                      ...  
wyoming  059       independent         26.0
                   republican        2658.0
         060       democrat          1260.0
                   independent          7.0
                   republican        2793.0
Name: votes, Length: 7480, dtype: float64

In [ ]:
%%time
winners_per_district = []
for state_col in state_house["state"].sort_values().unique():
  for distcol in state_house[state_house["state"] == state_col]["district"].unique():
      winners_per_district.append(test_house_district[state_col][distcol].idxmax())

CPU times: user 3.79 s, sys: 15.4 ms, total: 3.8 s
Wall time: 3.79 s


In [ ]:
from collections import Counter
Counter(winners_per_district)

Counter({'democrat': 1739, 'independent': 16, 'republican': 2002})

#### Check for unopposed elections

In [ ]:
unopposed_elections = []
for state in state_house["state"].sort_values().unique():
  for district in state_house[state_house["state"] == state]["district"].unique():
    if test_house_district[state][district].nunique() < 2:
      unopposed_elections.append([state, district])

In [ ]:
print(f"There are {len(unopposed_elections)} unopposed elections.")

There are 848 unopposed elections.


In [ ]:
state_house["state"].sort_values().unique()

array(['alaska', 'arkansas', 'california', 'colorado', 'connecticut',
       'delaware', 'florida', 'georgia', 'hawaii', 'idaho', 'illinois',
       'iowa', 'kansas', 'kentucky', 'maine', 'massachusetts', 'michigan',
       'minnesota', 'missouri', 'montana', 'nevada', 'new hampshire',
       'new jersey', 'new mexico', 'new york', 'north carolina', 'ohio',
       'oklahoma', 'pennsylvania', 'rhode island', 'south carolina',
       'south dakota', 'tennessee', 'texas', 'utah', 'vermont',
       'virginia', 'washington', 'west virginia', 'wisconsin', 'wyoming'],
      dtype=object)

## Create new DataFrame

In [ ]:
test_house_district

state    district  party_detailed
alaska   001       democrat          3027.0
                   republican        3769.0
         002       democrat          1565.0
                   republican        3631.0
         003       republican        7001.0
                                      ...  
wyoming  059       independent         26.0
                   republican        2658.0
         060       democrat          1260.0
                   independent          7.0
                   republican        2793.0
Name: votes, Length: 7480, dtype: float64

### Create target, district and state variable 

In [ ]:
target_list = []
district_list = []
state_list = []

for state in state_house["state"].sort_values().unique():
  # Makes list of all districts
  district_list.extend(state_house[state_house["state"] == state]["district"].unique().tolist())
  
  # Makes list of all states
  multiplier = state_house[state_house["state"] == state]["district"].nunique()
  state_list.extend([state] * multiplier)
  
  # Makes list of target variable (-> whether republican, democrat or indpendent)
  for district in state_house[state_house["state"] == state]["district"].unique():
    target_list.append(test_house_district[state][district].idxmax())

In [ ]:
len(target_list), len(district_list), len(state_list)

(3757, 3757, 3757)

In [ ]:
from collections import Counter
Counter(target_list)

Counter({'democrat': 1739, 'independent': 16, 'republican': 2002})

In [ ]:
Counter(state_list)

Counter({'alaska': 40,
         'arkansas': 97,
         'california': 80,
         'colorado': 65,
         'connecticut': 151,
         'delaware': 41,
         'florida': 96,
         'georgia': 180,
         'hawaii': 34,
         'idaho': 70,
         'illinois': 118,
         'iowa': 100,
         'kansas': 125,
         'kentucky': 100,
         'maine': 151,
         'massachusetts': 160,
         'michigan': 110,
         'minnesota': 134,
         'missouri': 163,
         'montana': 100,
         'nevada': 42,
         'new hampshire': 56,
         'new jersey': 1,
         'new mexico': 25,
         'new york': 150,
         'north carolina': 120,
         'ohio': 99,
         'oklahoma': 37,
         'pennsylvania': 203,
         'rhode island': 75,
         'south carolina': 124,
         'south dakota': 4,
         'tennessee': 99,
         'texas': 149,
         'utah': 75,
         'vermont': 58,
         'virginia': 1,
         'washington': 98,
         'west virgini

**Next Steps:**
* add office and year
* add Results of Oregon and Indiana
  * Indiana: https://ballotpedia.org/Indiana_House_of_Representatives_elections,_2020
  * Oregon: https://sos.oregon.gov/elections/Documents/results/november-general-2020.pdf
* Look for data to predict the target on: https://data.census.gov/cedsci/advanced

In [ ]:
state_house.head()

,Unnamed: 0,precinct,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,district,magnitude,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check
0,16156,01-446 aurora,state house,democrat,democrat,election day,248.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
1,16157,01-446 aurora,state house,republican,republican,election day,495.0,NaN,NaN,district 1,2001.0,"barton s ""bart"" lebon",001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
2,16158,01-455 fairbanks no. 1,state house,democrat,democrat,election day,45.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
3,16159,01-455 fairbanks no. 1,state house,republican,republican,election day,90.0,NaN,NaN,district 1,2001.0,"barton s ""bart"" lebon",001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False
4,16160,01-465 fairbanks no. 2,state house,democrat,democrat,election day,88.0,NaN,NaN,district 1,2001.0,christopher quist,001,1,state,2020,gen,alaska,False,False,ak,2,94,81,2020-11-03,False


In [ ]:
def create_dataframe(state_list, district_list, target_list):
  df = pd.DataFrame({
    "state": state_list,
    "district": district_list,
    "office": ["state house"] * len(state_list),
    "year": [2020] * len(state_list),
    "target": target_list})
  return df

In [ ]:
df = create_dataframe(state_list, district_list, target_list)
df.head()

,state,district,office,year,target
0,alaska,001,state house,2020,republican
1,alaska,002,state house,2020,republican
2,alaska,003,state house,2020,republican
3,alaska,004,state house,2020,democrat
4,alaska,005,state house,2020,democrat


### Adding Indiana and Oregon to the DataFrame

In [ ]:
df.target.value_counts()

republican     2002
democrat       1739
independent      16
Name: target, dtype: int64

In [ ]:
# Indiana
state_indiana = ["indiana"] * 100
district_indiana = np.arange(1, 101)
# republicans: 0, democrats: 1, independents: 2
target_indiana = [1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 
                  0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 
                  1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
                  0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 
                  0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]

indiana = create_dataframe(state_indiana, district_indiana, target_indiana)
indiana.head()

,state,district,office,year,target
0,indiana,1,state house,2020,1
1,indiana,2,state house,2020,1
2,indiana,3,state house,2020,1
3,indiana,4,state house,2020,0
4,indiana,5,state house,2020,0


In [ ]:
# Oregon
state_oregon = ["oregon"] * 60
district_oregon = np.arange(1, 61)
target_oregon = [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 
                 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 
                 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]

oregon = create_dataframe(state_oregon, district_oregon, target_oregon)
oregon.head()

,state,district,office,year,target
0,oregon,1,state house,2020,0
1,oregon,2,state house,2020,0
2,oregon,3,state house,2020,0
3,oregon,4,state house,2020,0
4,oregon,5,state house,2020,1


In [ ]:
# Add these two states to df
df = pd.concat([df, indiana, oregon])
df.tail()

,state,district,office,year,target
55,oregon,56,state house,2020,0
56,oregon,57,state house,2020,0
57,oregon,58,state house,2020,0
58,oregon,59,state house,2020,0
59,oregon,60,state house,2020,0


In [ ]:
### Change target variable to numeric
target_values = {"republican": 0, "democrat": 1, "independent": 2}
df.replace({"target": target_values}, inplace=True)
df.head()

,state,district,office,year,target
0,alaska,001,state house,2020,0
1,alaska,002,state house,2020,0
2,alaska,003,state house,2020,0
3,alaska,004,state house,2020,1
4,alaska,005,state house,2020,1


In [ ]:
# Sort dataframe by state names and save it to csv
df.sort_values(["state", "district"], inplace=True)
df.to_csv("drive/MyDrive/US Elections/Archive/target_ready_2020.csv", index=False)

# Final Cleanings

In this part the data from 2020 has to be cleaned so that it matches the one from 2016.



In [ ]:
import pandas as pd
import numpy as np

data16 = pd.read_csv("drive/MyDrive/US Elections/data_target_2016.csv")
data20 = pd.read_csv("drive/MyDrive/US Elections/Archive/target_ready_2020.csv")

In [ ]:
data16.head()

,district,democrat,republican,other,state,year,target
0,1,1.0,0.0,0.0,alaska,2016,1
1,10,2021.0,5901.0,0.0,alaska,2016,0
2,11,0.0,5752.0,2681.0,alaska,2016,0
3,12,2061.0,5597.0,949.0,alaska,2016,0
4,13,0.0,1.0,0.0,alaska,2016,0


In [ ]:
data20.head()

,state,district,office,year,target
0,alaska,001,state house,2020,0
1,alaska,002,state house,2020,0
2,alaska,003,state house,2020,0
3,alaska,004,state house,2020,1
4,alaska,005,state house,2020,1


In [ ]:
data20.shape

(3917, 5)

In [ ]:
# Idaho, South Dakota, Washington: all districts have more than 1 seat -> delete
# New Hampshire: because of different reasons just about 30 of total 400 seats available -> delete
data20.groupby("state")["district"].unique()

state
alaska            [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
arkansas          [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
california        [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
colorado          [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
connecticut       [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
delaware          [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
florida           [001, 002, 003, 004, 006, 007, 009, 010, 011, ...
georgia           [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
hawaii            [001, 002, 003, 004, 005, 009, 010, 011, 013, ...
idaho             [001, seat a, 001, seat b, 002, seat a, 002, s...
illinois          [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
indiana           [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
iowa              [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
kansas            [001, 002, 003, 004, 005, 006, 007, 008, 009, ...
kentucky          [001, 002, 003, 004, 005

## Tasks to solve
* Delete erroneous states✅
* get 0 from number away✅
* replace vermont districts with complete names✅
* Massachusetts: Barnstable etc. "and" to "&" ✅
* fill missing districts for arkansas (3 missing), florida (24), hawaii (17), new mexico (45), oklahoma (64), texas (1)✅

In [ ]:
# Delete erroneous states
drop_index = data20[data20.state.isin(["idaho", "new hampshire", "new jersey", "south dakota", "virginia", "washington"])].index
data20 = data20.drop(drop_index, axis=0)

In [ ]:
# Get 0 from number away
def clean_num_districts(string):
  if string.split("0")[-1] != "":
    return string.split("0")[-1]
  elif string.split("0")[-1] == "":
    return string.split("0")[-2] + "0"
  else:
    pass

In [ ]:
# Need to leave Massachusetts out as 10th, 20th etc in their district names would lead to complications
data20.loc[data20[data20.state != "massachusetts"].index, "district"] = data20[data20.state != "massachusetts"].district.apply(clean_num_districts)
data20.district.tail()

3912    56
3913    57
3914    58
3915    59
3916    60
Name: district, dtype: object

In [ ]:
# Get 0 from number away

def clean_num_districts(string):
  if string.startswith("00"):
    return string[-1]
  elif string.startswith("0"):
    return string[-2:]
  else:
    return string

# Need to leave Massachusetts out as 10th, 20th etc in their district names would lead to complications
data20.loc[data20[data20.state != "massachusetts"].index, "district"] = data20[data20.state != "massachusetts"].district.apply(clean_num_districts)
data20.district.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '47',
       '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
       '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80',
       '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91',
       '92', '93', '94', '95', '96', '97', '98', '99', '0', '45', '46',
       '110', '111', '112', '113', '114', '115', '116', '117', '118',
       '119', '120', '121', '122', '123', '124', '125', '126', '127',
       '128', '129', '130', '131', '132', '133', '134', '135', '136',
       '137', '138', '139', '140', '141', '142', '143', '144', '145',
       '146', '147', '148', '149', '150', '151', '152', '153', '154

In [ ]:
# Replace vermont districts with complete names
data20[data20.state == "vermont"].district.unique()[:10]

array(['add-2', 'add-5', 'add-rut', 'ben-1', 'ben-3', 'ben-rut', 'cal-1',
       'cal-2', 'cal-was', 'chi-1'], dtype=object)

In [ ]:
vermont_replacer = {
    "add": "addison",
    "rut": "rutland",
    "ben": "bennington",
    "cal": "caledonia",
    "was": "washington",
    "chi": "chittenden",
    "esx": "essex",
    "orl": "orleans",
    "fra": "franklin",
    "lam": "lamoille",
    "wdr": "windsor",
    "wdh": "windham",
    "ora": "orange"
}

In [ ]:
# Create separate series for all the parts of district names
sub1 = data20[data20.state == "vermont"].district.str.split("-").str[0]
sub2 = data20[data20.state == "vermont"].district.str.split("-").str[1]
sub3 = data20[data20.state == "vermont"].district.str.split("-").str[2]
sub3 = sub3.replace(np.nan, "") # replace NA's with "" to be able to join them together again in next step

In [ ]:
vermont_district = sub1.replace(vermont_replacer)+"-"+sub2.replace(vermont_replacer)+"-"+sub3.replace(vermont_replacer)
vermont_district_final = vermont_district.str.rstrip("-")

In [ ]:
# Actually replacing district column
data20.loc[data20[data20.state == "vermont"].index, "district"] = vermont_district_final
data20[data20.state == "vermont"].district.unique()

array(['addison-2', 'addison-5', 'addison-rutland', 'bennington-1',
       'bennington-3', 'bennington-rutland', 'caledonia-1', 'caledonia-2',
       'caledonia-washington', 'chittenden-1', 'chittenden-4-1',
       'chittenden-4-2', 'chittenden-5-1', 'chittenden-5-2',
       'chittenden-6-2', 'chittenden-6-6', 'chittenden-7-1',
       'chittenden-7-2', 'chittenden-7-3', 'chittenden-7-4',
       'chittenden-8-3', 'essex-caledonia', 'essex-caledonia-orleans',
       'franklin-1', 'franklin-2', 'franklin-3-2', 'franklin-6',
       'franklin-7', 'lamoille-1', 'lamoille-3', 'orange-2',
       'orange-caledonia', 'orleans-lamoille', 'rutland-1', 'rutland-4',
       'rutland-5-1', 'rutland-5-2', 'rutland-5-3', 'rutland-5-4',
       'rutland-bennington', 'rutland-windsor-1', 'rutland-windsor-2',
       'washington-5', 'washington-6', 'windham-1', 'windham-2-1',
       'windham-2-2', 'windham-2-3', 'windham-5', 'windham-6',
       'windham-bennington', 'windham-bennington-windsor', 'windsor-2',

In [ ]:
# Clean Massachusetts district --> problem with the 0 deleting -> deleted 0 of massachusetts district
data20.loc[data20[data20.district == "barnstable, dukes, and nantucket"].index, "district"] = "barnstable, dukes & nantucket"

In [ ]:
# Incomplete: arkansas (3 missing), florida (24), hawaii (17), new mexico (45), oklahoma (64), texas (1)
## Arkansas: 12, 45, 46
## Florida: 5, 8, 13, 14, 20, 43, 45, 46, 61, 63, 70, 94-100, 102, 107, 108, 109, 113, 117
## New Mexico: 26 - 70
## Texas: 98
## Hawaii and Oklahoma: In the missing districts there were no general elections
data20.groupby("state")["district"].nunique()

state
alaska             40
arkansas           97
california         80
colorado           65
connecticut       142
delaware           41
florida            92
georgia           171
hawaii             34
illinois          109
indiana           100
iowa              100
kansas            116
kentucky          100
maine             142
massachusetts     160
michigan          101
minnesota         124
missouri          154
montana           100
nevada             42
new mexico         25
new york          141
north carolina    111
ohio               99
oklahoma           37
oregon             60
pennsylvania      190
rhode island       75
south carolina    115
tennessee          99
texas             140
utah               75
vermont            58
west virginia      67
wisconsin          99
wyoming            60
Name: district, dtype: int64

In [ ]:
# Arkansas
arkansas_state = ["arkansas"] * 3
arkansas_district = ["12", "45", "46"]
arkansas_target = [0, 0, 0]
arkansas_office = ["state house"] * 3
arkansas_year = ["2020"] * 3

arkansas = pd.DataFrame({
    "state": arkansas_state,
    "district": arkansas_district,
    "office": arkansas_office,
    "year": arkansas_year,
    "target": arkansas_target
})
arkansas.head()

,state,district,office,year,target
0,arkansas,12,state house,2020,0
1,arkansas,45,state house,2020,0
2,arkansas,46,state house,2020,0


In [ ]:
# Florida
florida_state = ["florida"] * 24
florida_district = ["5", "8", "13", "14", "20", "43", "45", "46", "61", "63", "70", "94", "95", "96", "97", "98", "99", 
                    "100", "102", "107", "108", "109", "113", "117"]
florida_target = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ]
florida_office = ["state house"] * 24
florida_year = ["2020"] * 24

florida = pd.DataFrame({
    "state": florida_state,
    "district": florida_district,
    "office": florida_office,
    "year": florida_year,
    "target": florida_target
})
florida.head()

,state,district,office,year,target
0,florida,5,state house,2020,0
1,florida,8,state house,2020,1
2,florida,13,state house,2020,1
3,florida,14,state house,2020,1
4,florida,20,state house,2020,1


In [ ]:
# New Mexico
new_mexico_state = ["new mexico"] * 45
new_mexico_district = np.arange(26, 71).astype("str")
new_mexico_target = [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 
                     0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 
                     0, 0, 0, 1, 0, 0, 1, 1, 1]
new_mexico_office = ["state house"] * 45
new_mexico_year = ["2020"] * 45

new_mexico = pd.DataFrame({
    "state": new_mexico_state,
    "district": new_mexico_district,
    "office": new_mexico_office,
    "year": new_mexico_year,
    "target": new_mexico_target
})
new_mexico.head()

,state,district,office,year,target
0,new mexico,26,state house,2020,1
1,new mexico,27,state house,2020,1
2,new mexico,28,state house,2020,1
3,new mexico,29,state house,2020,1
4,new mexico,30,state house,2020,1


In [ ]:
# Texas
texas_state = ["texas"]
texas_district = ["98"]
texas_target = [0]
texas_office = ["state house"]
texas_year = ["2020"]

texas = pd.DataFrame({
    "state": texas_state,
    "district": texas_district,
    "office": texas_office,
    "year": texas_year,
    "target": texas_target
})
texas.head()

,state,district,office,year,target
0,texas,98,state house,2020,0


In [ ]:
data20_final = pd.concat([data20, arkansas, florida, new_mexico, texas], axis=0).sort_values(by=["state", "district"]).reset_index(drop=True)

In [ ]:
data20_final.shape

(3760, 5)

In [ ]:
data20_final.tail()

,state,district,office,year,target
3755,wyoming,6,state house,2020,0
3756,wyoming,60,state house,2020,0
3757,wyoming,7,state house,2020,0
3758,wyoming,8,state house,2020,0
3759,wyoming,9,state house,2020,0


In [ ]:
# Save to CSV
data20_final.to_csv("drive/MyDrive/US Elections/data_target_2020.csv", index=False)